Installs

In [1]:
! pip install rdkit pandas numpy matplotlib seaborn scikit-learn jupyter tqdm
! pip install rdkit

Imports

In [2]:
import pandas as pd
import numpy as np
import warnings
from rdkit import Chem
from rdkit.Chem import AllChem
from rdkit import DataStructs
from rdkit.Chem import Draw


CORE FUNCTIONS FOR CHEMICAL SIMILARITY

In [3]:

def load_molecules_from_sdf(sdf_path):
    """
    Load drug molecules from SDF file
    """
    mols = []
    suppl = Chem.SDMolSupplier(sdf_path)
    
    for idx, mol in enumerate(suppl):
        if mol is not None:
            mol.SetProp("_Index", str(idx))
            mols.append(mol)
        else:
            print(f"Warning: Failed to load molecule at index {idx}")
    
    print(f"✓ Loaded {len(mols)} valid molecules from {sdf_path}")
    return mols

def load_molecules_from_smiles(smiles_list, names=None):
    """
    Load drug molecules from SMILES strings
    """
    mols = []
    
    for idx, smiles in enumerate(smiles_list):
        mol = Chem.MolFromSmiles(smiles)
        if mol is not None:
            if names and idx < len(names):
                mol.SetProp("_Name", names[idx])
            mol.SetProp("_Index", str(idx))
            mols.append(mol)
        else:
            print(f"Warning: Invalid SMILES at index {idx}: {smiles}")
    
    print(f"✓ Loaded {len(mols)} valid molecules from SMILES list")
    return mols

def calculate_fingerprint(mol, fp_type='Morgan', radius=2, n_bits=1024):
    """
    Calculate molecular fingerprint
    """
    if fp_type == 'Morgan':
        return AllChem.GetMorganFingerprintAsBitVect(mol, radius=radius, nBits=n_bits)
    elif fp_type == 'AtomPair':
        return Chem.rdmolops.GetAtomPairFingerprintAsBitVect(mol)
    elif fp_type == 'Topological':
        return Chem.RDKFingerprint(mol)
    else:
        raise ValueError("fp_type must be 'Morgan', 'AtomPair', or 'Topological'")

def calculate_tanimoto_similarity(fp1, fp2):
    """
    Calculate Tanimoto coefficient between two fingerprints
    """
    return DataStructs.TanimotoSimilarity(fp1, fp2)

PAIRWISE SIMILARITY ANALYSIS

In [4]:
def calculate_similarity_matrix(mols, fp_type='Morgan'):
    """
    Calculate pairwise Tanimoto similarity matrix for all molecules
    """
    n_mols = len(mols)
    fingerprints = [calculate_fingerprint(mol, fp_type) for mol in mols]
    
    similarity_matrix = np.zeros((n_mols, n_mols))
    
    print("Calculating pairwise similarities...")
    for i in range(n_mols):
        for j in range(i, n_mols):
            if i == j:
                similarity_matrix[i, j] = 1.0
            else:
                sim = calculate_tanimoto_similarity(fingerprints[i], fingerprints[j])
                similarity_matrix[i, j] = sim
                similarity_matrix[j, i] = sim
    
    print(f"✓ Calculated similarity matrix for {n_mols} molecules")
    return similarity_matrix, fingerprints

def find_similar_pairs(similarity_matrix, mols=None, threshold=0.3):
    """
    Find all pairs of molecules with similarity above threshold
    """
    n = similarity_matrix.shape[0]
    similar_pairs = []
    
    for i in range(n):
        for j in range(i + 1, n):
            sim = similarity_matrix[i, j]
            if sim >= threshold:
                if mols:
                    name_i = mols[i].GetProp("_Name") if mols[i].HasProp("_Name") else f"Compound_{i}"
                    name_j = mols[j].GetProp("_Name") if mols[j].HasProp("_Name") else f"Compound_{j}"
                    similar_pairs.append({
                        'index_i': i,
                        'index_j': j,
                        'name_i': name_i,
                        'name_j': name_j,
                        'similarity': sim
                    })
                else:
                    similar_pairs.append((i, j, sim))
    
    # Sort by similarity (highest first)
    if mols and similar_pairs:
        similar_pairs.sort(key=lambda x: x['similarity'], reverse=True)
    elif similar_pairs:
        similar_pairs.sort(key=lambda x: x[2], reverse=True)
    
    print(f"Found {len(similar_pairs)} pairs with similarity ≥ {threshold}")
    return similar_pairs

def analyze_similarity_distribution(similarity_matrix):
    """
    Generate statistics about the similarity distribution
    """
    n = similarity_matrix.shape[0]
    similarities = []
    
    # Get upper triangle (excluding diagonal)
    for i in range(n):
        for j in range(i + 1, n):
            similarities.append(similarity_matrix[i, j])
    
    similarities = np.array(similarities)
    
    stats = {
        'n_pairs': len(similarities),
        'mean': similarities.mean(),
        'median': np.median(similarities),
        'std': similarities.std(),
        'min': similarities.min(),
        'max': similarities.max(),
        'q25': np.percentile(similarities, 25),
        'q75': np.percentile(similarities, 75)
    }
    
    # Count pairs above different thresholds
    thresholds = [0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9]
    for thresh in thresholds:
        count = np.sum(similarities >= thresh)
        percentage = (count / len(similarities)) * 100
        stats[f'pairs_above_{thresh}'] = count
        stats[f'percent_above_{thresh}'] = percentage
    
    return stats, similarities

VISUALIZATION FUNCTIONS

In [5]:
def visualize_molecules(mols, indices=None, mols_per_row=4, sub_img_size=(200, 200)):
    """
    Display molecules in a grid
    """
    if indices is None:
        indices = range(min(12, len(mols)))  # Show max 12 molecules
    
    mols_to_show = [mols[i] for i in indices if i < len(mols)]
    
    legends = []
    for idx in indices:
        if idx < len(mols):
            mol = mols[idx]
            if mol.HasProp("_Name"):
                legends.append(mol.GetProp("_Name"))
            else:
                legends.append(f"Compound_{idx}")
    
    img = Draw.MolsToGridImage(mols_to_show,
                              molsPerRow=mols_per_row,
                              subImgSize=sub_img_size,
                              legends=legends)
    return img

def visualize_similar_pairs(mols, similar_pairs, max_pairs=5):
    """
    Visualize similar molecule pairs
    """
    if not similar_pairs:
        print("No similar pairs to visualize")
        return
    
    num_to_show = min(max_pairs, len(similar_pairs))
    print(f"\nTop {num_to_show} similar compound pairs:")
    
    images = []
    for idx in range(num_to_show):
        pair = similar_pairs[idx]
        if isinstance(pair, dict):
            i, j = pair['index_i'], pair['index_j']
            sim = pair['similarity']
            name_i, name_j = pair['name_i'], pair['name_j']
        else:
            i, j, sim = pair
            name_i = f"Compound_{i}"
            name_j = f"Compound_{j}"
        
        print(f"Pair {idx+1}: {name_i} and {name_j}, Similarity: {sim:.3f}")
        
        img = Draw.MolsToGridImage([mols[i], mols[j]],
                                  molsPerRow=2,
                                  subImgSize=(300, 300),
                                  legends=[f"{name_i} (Sim: {sim:.3f})", 
                                           f"{name_j} (Sim: {sim:.3f})"])
        images.append(img)
    
    return images

def plot_similarity_histogram(similarities, threshold=0.3, bins=50):
    """
    Create histogram of similarity distribution
    """
    import matplotlib.pyplot as plt
    
    plt.figure(figsize=(10, 6))
    
    # Plot histogram
    plt.hist(similarities, bins=bins, alpha=0.7, color='skyblue', edgecolor='black')
    
    # Add threshold line
    plt.axvline(x=threshold, color='red', linestyle='--', linewidth=2, 
                label=f'Threshold = {threshold}')
    
    plt.title('Distribution of Tanimoto Similarities', fontsize=14)
    plt.xlabel('Tanimoto Similarity Coefficient', fontsize=12)
    plt.ylabel('Frequency', fontsize=12)
    plt.grid(True, alpha=0.3)
    plt.legend()
    
    # Add text with statistics
    stats_text = f'Total pairs: {len(similarities):,}\n'
    stats_text += f'Mean: {similarities.mean():.3f}\n'
    stats_text += f'Above threshold: {np.sum(similarities >= threshold):,}'
    
    plt.text(0.7, 0.95, stats_text, transform=plt.gca().transAxes,
             fontsize=10, verticalalignment='top',
             bbox=dict(boxstyle='round', facecolor='wheat', alpha=0.5))
    
    plt.tight_layout()
    return plt

FILTERING AND DATASET MANAGEMENT

In [6]:
def remove_similar_compounds(mols, fingerprints, threshold=0.3, method='first'):
    """
    Remove structurally similar compounds from dataset
    
    Parameters:
    -----------
    mols : list of RDKit molecules
    fingerprints : list of fingerprints
    threshold : similarity cutoff
    method : 'first' (keep first encountered) or 'most_diverse' (keep most diverse)
    """
    n = len(mols)
    to_keep = [True] * n
    removed_indices = []
    
    print(f"Filtering compounds with similarity ≥ {threshold}...")
    
    for i in range(n):
        if not to_keep[i]:
            continue
        
        for j in range(i + 1, n):
            if not to_keep[j]:
                continue
            
            sim = calculate_tanimoto_similarity(fingerprints[i], fingerprints[j])
            if sim >= threshold:
                if method == 'most_diverse':
                    # Keep the compound with lower average similarity to others
                    # (This is a simplified approach - more sophisticated methods exist)
                    to_keep[j] = False
                else:  # 'first' method
                    to_keep[j] = False
                removed_indices.append(j)
    
    # Create filtered lists
    filtered_mols = [mols[i] for i in range(n) if to_keep[i]]
    filtered_fingerprints = [fingerprints[i] for i in range(n) if to_keep[i]]
    
    # Statistics
    kept_indices = [i for i in range(n) if to_keep[i]]
    
    print(f"✓ Original dataset: {n} compounds")
    print(f"✓ After filtering: {len(filtered_mols)} compounds")
    print(f"✓ Removed: {len(removed_indices)} compounds")
    print(f"✓ Kept indices: {kept_indices[:10]}{'...' if len(kept_indices) > 10 else ''}")
    
    return {
        'molecules': filtered_mols,
        'fingerprints': filtered_fingerprints,
        'kept_indices': kept_indices,
        'removed_indices': removed_indices
    }

def save_filtered_molecules(mols, output_path='filtered_compounds.sdf'):
    """
    Save filtered molecules to SDF file
    """
    writer = Chem.SDWriter(output_path)
    
    for idx, mol in enumerate(mols):
        # Ensure molecule has required properties
        if not mol.HasProp("_Name"):
            mol.SetProp("_Name", f"Compound_{idx}")
        writer.write(mol)
    
    writer.close()
    print(f"✓ Saved {len(mols)} molecules to {output_path}")

def export_similarity_results(similarity_matrix, mols, output_csv='similarity_results.csv'):
    """
    Export similarity results to CSV file
    """
    n = similarity_matrix.shape[0]
    results = []
    
    for i in range(n):
        for j in range(i + 1, n):
            name_i = mols[i].GetProp("_Name") if mols[i].HasProp("_Name") else f"Compound_{i}"
            name_j = mols[j].GetProp("_Name") if mols[j].HasProp("_Name") else f"Compound_{j}"
            
            results.append({
                'Compound_1': name_i,
                'Compound_2': name_j,
                'Tanimoto_Similarity': similarity_matrix[i, j]
            })
    
    df = pd.DataFrame(results)
    df = df.sort_values('Tanimoto_Similarity', ascending=False)
    df.to_csv(output_csv, index=False)
    
    print(f"✓ Exported {len(df)} similarity pairs to {output_csv}")
    return df

EXAMPLE USAGE AND MAIN PIPELINE

In [7]:
def example_from_sdf():
    """
    Example pipeline for SDF files (as described in the paper)
    """
    print("=" * 60)
    print("EXAMPLE 1: SDF FILE PIPELINE")
    print("=" * 60)
    
    # 1. Load molecules
    sdf_file = "your_drugs.sdf"  # Change this to your SDF file
    mols = load_molecules_from_sdf(sdf_file)
    
    # 2. Calculate fingerprints and similarity matrix
    similarity_matrix, fingerprints = calculate_similarity_matrix(mols, fp_type='AtomPair')
    
    # 3. Analyze similarity distribution
    stats, similarities = analyze_similarity_distribution(similarity_matrix)
    print("\nSimilarity Statistics:")
    for key, value in stats.items():
        if not key.startswith('pairs_above') and not key.startswith('percent_above'):
            print(f"  {key}: {value}")
    
    # 4. Find similar pairs
    similar_pairs = find_similar_pairs(similarity_matrix, mols, threshold=0.3)
    
    # 5. Visualize top similar pairs
    if similar_pairs:
        visualize_similar_pairs(mols, similar_pairs, max_pairs=3)
    
    # 6. Filter similar compounds
    filtered_data = remove_similar_compounds(mols, fingerprints, threshold=0.3)
    
    # 7. Save results
    save_filtered_molecules(filtered_data['molecules'], 'filtered_drugs.sdf')
    export_similarity_results(similarity_matrix, mols, 'drug_similarities.csv')
    
    return {
        'molecules': mols,
        'similarity_matrix': similarity_matrix,
        'similar_pairs': similar_pairs,
        'filtered_data': filtered_data,
        'stats': stats
    }

def example_from_smiles():
    """
    Example pipeline for SMILES strings
    """
    print("=" * 60)
    print("EXAMPLE 2: SMILES PIPELINE")
    print("=" * 60)
    
    # Example SMILES for cardiovascular drugs
    drug_smiles = [
        'CC(=O)OC1=CC=CC=C1C(=O)O',  # Aspirin
        'CC1=C(C(=CC=C1)NC(=O)C2=CC=C(C=C2)Cl)OC',  # Atenolol
        'CC(C)CC(=O)N1CCC(CC1)C2=CC(=C(C=C2)OC)C(=O)O',  # Atorvastatin
        'CN1C=NC2=C1C(=O)N(C(=O)N2C)C',  # Caffeine (control)
    ]
    
    drug_names = ['Aspirin', 'Atenolol', 'Atorvastatin', 'Caffeine']
    
    # 1. Load from SMILES
    mols = load_molecules_from_smiles(drug_smiles, drug_names)
    
    # 2. Calculate similarities
    similarity_matrix, fingerprints = calculate_similarity_matrix(mols)
    
    # 3. Find similar pairs
    similar_pairs = find_similar_pairs(similarity_matrix, mols, threshold=0.3)
    
    # 4. Visualize
    print("\nAll molecules:")
    img_all = visualize_molecules(mols, mols_per_row=2)
    
    if similar_pairs:
        print("\nSimilar pairs:")
        visualize_similar_pairs(mols, similar_pairs)
    
    return {
        'molecules': mols,
        'similarity_matrix': similarity_matrix,
        'similar_pairs': similar_pairs
    }

def quick_similarity_check(smiles1, smiles2, name1="Drug1", name2="Drug2"):
    """
    Quickly check similarity between two drugs
    """
    print(f"Comparing {name1} and {name2}...")
    
    mol1 = Chem.MolFromSmiles(smiles1)
    mol2 = Chem.MolFromSmiles(smiles2)
    
    if mol1 is None or mol2 is None:
        print("Error: Invalid SMILES string")
        return None
    
    fp1 = calculate_fingerprint(mol1, 'Morgan')
    fp2 = calculate_fingerprint(mol2, 'Morgan')
    
    similarity = calculate_tanimoto_similarity(fp1, fp2)
    
    print(f"Tanimoto similarity between {name1} and {name2}: {similarity:.3f}")
    
    # Visualize
    img = Draw.MolsToGridImage([mol1, mol2],
                              molsPerRow=2,
                              subImgSize=(300, 300),
                              legends=[f"{name1}", f"{name2}"])
    
    return similarity, img

if __name__ == "__main__":
    # Run the examples
    print("Chemical Structural Similarity Analysis")
    print("Based on: Computational models for prediction of adverse cardiovascular drug reactions")
    
    # Example 1: Quick comparison of two drugs
    print("\n" + "="*50)
    print("QUICK COMPARISON EXAMPLE")
    print("="*50)
    
    aspirin_smiles = "CC(=O)OC1=CC=CC=C1C(=O)O"
    ibuprofen_smiles = "CC(C)CC1=CC=C(C=C1)C(C)C(=O)O"
    
    quick_similarity_check(aspirin_smiles, ibuprofen_smiles, 
                          "Aspirin", "Ibuprofen")
    
    # Uncomment to run full examples:
    # example_from_smiles()
    # results = example_from_sdf()

Chemical Structural Similarity Analysis
Based on: Computational models for prediction of adverse cardiovascular drug reactions

QUICK COMPARISON EXAMPLE
Comparing Aspirin and Ibuprofen...
Tanimoto similarity between Aspirin and Ibuprofen: 0.195


[14:34:25] DEPRECATION WARNING: please use MorganGenerator
[14:34:25] DEPRECATION WARNING: please use MorganGenerator


Process data

In [ ]:
def run_test(csv_path="../Data/drugs_with_sider_labels.csv"):
    
    print("Running test on drug metadata dataset...\n")
    
    # Load dataset
    df = pd.read_csv(csv_path)

    required_cols = [
        "Parent Molecule", "Name", "InChIKey",
        "Drug Type", "Phase", "Withdrawn Flag"
    ]
    
    missing = [c for c in required_cols if c not in df.columns]
    if missing:
        raise ValueError(f"Missing required columns: {missing}")

    inchikey_missing = df["InChIKey"].isna().sum()

    print("Dataset statistics:")
    print(df["Drug Type"].value_counts().head())

    return True

run


Running test on drug metadata dataset...

Dataset statistics:
Drug Type
1:Synthetic Small Molecule    1890
9:Inorganic                     28
10:Polymer                       6
Name: count, dtype: int64


True

In [13]:
import pandas as pd
from rdkit import Chem, DataStructs
from rdkit.Chem import AllChem

def filter_diverse_molecules(
    csv_path,
    output_csv,
    smiles_col="Smiles",
    similarity_threshold=0.3,
    radius=2,
    n_bits=2048
):
    """
    Filters molecules so that all remaining molecules are structurally different
    according to a Tanimoto similarity threshold.
    """

    df = pd.read_csv(csv_path)

    if smiles_col not in df.columns:
        raise ValueError(f"Column '{smiles_col}' not found. Structural similarity requires SMILES.")

    print(f"Loaded {len(df)} molecules")

    # Convert SMILES → RDKit molecules
    mols = []
    valid_indices = []

    for i, smi in enumerate(df[smiles_col]):
        if pd.isna(smi):
            continue
        mol = Chem.MolFromSmiles(smi)
        if mol:
            mols.append(mol)
            valid_indices.append(i)

    df = df.iloc[valid_indices].reset_index(drop=True)
    print(f"Valid molecules with SMILES: {len(df)}")

    # Compute fingerprints
    fps = [
        AllChem.GetMorganFingerprintAsBitVect(m, radius, nBits=n_bits)
        for m in mols
    ]

    selected_indices = []
    selected_fps = []

    for i, fp in enumerate(fps):
        if not selected_fps:
            selected_indices.append(i)
            selected_fps.append(fp)
            continue

        similarities = DataStructs.BulkTanimotoSimilarity(fp, selected_fps)
        max_sim = max(similarities)

        if max_sim < similarity_threshold:
            selected_indices.append(i)
            selected_fps.append(fp)

    diverse_df = df.iloc[selected_indices].reset_index(drop=True)

    print(f"Selected {len(diverse_df)} diverse molecules")
    print(f"Similarity threshold: {similarity_threshold}")

    # Save to CSV
    diverse_df.to_csv(output_csv, index=False)
    print(f"✓ Saved to {output_csv}")

    return diverse_df

diverse_df = filter_diverse_molecules(
    csv_path="../Data/drugs_with_sider_labels.csv",
    output_csv="../Data/diverse_molecules.csv",
    smiles_col="InChIKey",
    similarity_threshold=0.3
)


Loaded 1924 molecules
Valid molecules with SMILES: 0
Selected 0 diverse molecules
Similarity threshold: 0.3
✓ Saved to ../Data/diverse_molecules.csv


[14:41:26] SMILES Parse Error: syntax error while parsing: SPPTWHFVYKCNNK-FQEVSTJZSA-N
[14:41:26] SMILES Parse Error: check for mistakes around position 4:
[14:41:26] SPPTWHFVYKCNNK-FQEVSTJZSA-N
[14:41:26] ~~~^
[14:41:26] SMILES Parse Error: Failed parsing SMILES 'SPPTWHFVYKCNNK-FQEVSTJZSA-N' for input: 'SPPTWHFVYKCNNK-FQEVSTJZSA-N'
[14:41:26] SMILES Parse Error: syntax error while parsing: FFINMCNLQNTKLU-UHFFFAOYSA-N
[14:41:26] SMILES Parse Error: check for mistakes around position 5:
[14:41:26] FFINMCNLQNTKLU-UHFFFAOYSA-N
[14:41:26] ~~~~^
[14:41:26] SMILES Parse Error: Failed parsing SMILES 'FFINMCNLQNTKLU-UHFFFAOYSA-N' for input: 'FFINMCNLQNTKLU-UHFFFAOYSA-N'
[14:41:26] SMILES Parse Error: syntax error while parsing: OIRFJRBSRORBCM-UHFFFAOYSA-N
[14:41:26] SMILES Parse Error: check for mistakes around position 3:
[14:41:26] OIRFJRBSRORBCM-UHFFFAOYSA-N
[14:41:26] ~~^
[14:41:26] SMILES Parse Error: Failed parsing SMILES 'OIRFJRBSRORBCM-UHFFFAOYSA-N' for input: 'OIRFJRBSRORBCM-UHFFFAOYS